# **Slide-seq lung DestVI v2 imputed gene expression DE analysis (ct-aware Harreman results)**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '/home/projects/nyosef/oier/new_destvi_updated/scvi-tools/src')

In [3]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scvi
from scvi.model import CondSCVI, DestVI
import torch
import destvi_utils
from scipy.stats import ks_2samp, zscore
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import gseapy
from plotnine import *

BASE_PATH = "/home/projects/nyosef/oier/Harreman_files/Slide_seq_lung"
ADATA_PATH = os.path.join(BASE_PATH, 'h5ads')
DATA_PATH = os.path.join(BASE_PATH, 'data')
MODELS_PATH = os.path.join(BASE_PATH, 'models')
PLOTS_PATH = os.path.join(BASE_PATH, 'plots')
SLIDE_SEQ_LUNG_DESTVI_ADATA_PATH = os.path.join(ADATA_PATH, 'Slide_seq_lung_DestVI_v2_adata.h5ad')
SLIDE_SEQ_LUNG_SC_MODEL_PATH = os.path.join(MODELS_PATH, 'Slide_seq_lung_DestVI_v2_sc')
SLIDE_SEQ_LUNG_ST_MODEL_PATH = os.path.join(MODELS_PATH, 'Slide_seq_lung_DestVI_v2_st')

msigdb_data_path = "/home/projects/nyosef/oier/MSigDB_data"

/home/projects/nyosef/oier/mambaforge/envs/new_destvi-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/projects/nyosef/oier/mambaforge/envs/new_destvi-env/lib/python3.11/site-packages/docrep/decorators.py:43: SyntaxWarning: 'unlabeled_category' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [4]:
def de_genes(
    st_model,
    mask,
    ct,
    threshold=0.0,
    st_adata=None,
    mask2=None,
    key=None,
    N_sample=10,
    pseudocount=0.01,
    key_proportions="proportions",
):
    """
    Function to compute differential expressed genes from generative model.
    For further reference check [Lopez22]_.

    Parameters
    ----------
    st_adata
        Spatial sequencing dataset with proportions in obsm[key_proportions]. If not provided uses data in st_model.
    st_model
        Trained destVI model
    mask
        Mask for subsetting the spots to condition 1 in differential expression.
    mask2
        Mask for subsetting the spots to condition 2 in differential expression (reference). If none, inverse of mask.
    ct
        Cell type for which differential expression is computed
    threshold
        Proportion threshold to subset to spots with this amount of cell type proportion
    key
        Key to store values in st_adata.uns[key]. If None returns pandas dataframe with DE results. Defaults to None
    N_sample
        N_samples drawn from generative model to simulate expression values.
    pseudocount
        Pseudocount added at computation of logFC. Increasing leads to lower logFC of lowly expressed genes.
    key_proportions
        Obsm key pointing to cell-type proportions.

    Returns
    -------
    res
        If key is None. Pandas dataframe containing results of differential expression.
        Dataframe columns are "log2FC", "pval", "score".
        If key is provided. mask, mask2 and de_results are stored in st_adata.uns[key]. Dictionary keys are
        "mask_active", "mask_rest", "de_results".

    """

    # get statistics
    if mask2 is None:
        mask2 = ~mask

    if st_adata is None:
        st_adata = st_model.adata
        st_adata.obsm[key_proportions] = st_model.get_proportions()
    else:
        if key_proportions not in st_adata.obsm:
            raise ValueError(
                f"Please provide cell type proportions in st_adata.obsm[{key_proportions}] and rerun."
            )

    if st_model.registry_["setup_args"]["layer"]:
        expression = st_adata.layers[st_model.registry_["setup_args"]["layer"]]
    else:
        expression = st_adata.X

    # mask = np.logical_and(mask, st_adata.obsm[key_proportions][ct] > threshold)
    # mask2 = np.logical_and(mask2, st_adata.obsm[key_proportions][ct] > threshold)

    avg_library_size = np.mean(np.sum(expression, axis=1).flatten())
    exp_px_r = st_model.module.px_r.detach().exp().cpu().numpy()
    imputations = st_model.get_scale_for_ct(ct).values
    mean = avg_library_size * imputations

    concentration = torch.tensor(avg_library_size * imputations / exp_px_r)
    rate = torch.tensor(1.0 / exp_px_r)

    # slice conditions
    N_mask = N_unmask = N_sample

    def simulation(mask_, N_mask_):
        # generate
        simulated = (
            torch.distributions.Gamma(concentration=concentration[mask_], rate=rate)
            .sample((N_mask_,))
            .cpu()
            .numpy()
        )
        simulated = np.log(simulated + 1)
        simulated = simulated.reshape((-1, simulated.shape[-1]))
        return simulated

    simulated_case = simulation(mask, N_mask)
    simulated_control = simulation(mask2, N_unmask)

    de = np.array(
        [
            ks_2samp(
                simulated_case[:, gene],
                simulated_control[:, gene],
                alternative="two-sided",
                mode="asymp",
            )
            for gene in range(simulated_control.shape[1])
        ]
    )
    lfc = np.log2(pseudocount + mean[mask].mean(0)) - np.log2(
        pseudocount + mean[mask2].mean(0)
    )
    res = pd.DataFrame(
        data=np.vstack([lfc, de[:, 0], de[:, 1]]),
        columns=st_adata.var.index,
        index=["log2FC", "score", "pval"],
    ).T

    # Store results in st_adata
    if key is not None:
        st_adata.uns[key] = {}
        st_adata.uns[key]["de_results"] = res.sort_values(by="score", ascending=False)
        st_adata.uns[key]["mask_active"] = mask
        st_adata.uns[key]["mask_rest"] = mask2
        return st_adata
    else:
        return res

In [5]:
# adata = sc.read_h5ad(os.path.join(ADATA_PATH, 'Slide_seq_lung_ct_Harreman_adata.h5ad'))
adata = sc.read_h5ad(f'{ADATA_PATH}/Slide_seq_lung_ct_Harreman_no_deconv_adata.h5ad')

In [6]:
st_adata = sc.read_h5ad(SLIDE_SEQ_LUNG_DESTVI_ADATA_PATH)

In [7]:
st_model = DestVI.load(SLIDE_SEQ_LUNG_ST_MODEL_PATH, st_adata)

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


INFO     File /home/projects/nyosef/oier/Harreman_files/Slide_seq_lung/models/Slide_seq_lung_DestVI_v2_st/model.pt 
         already downloaded                                                                                        


In [8]:
def get_ct_proportion_thershold(proportion_df):
    
    mean = proportion_df.mean()
    std = proportion_df.std()
    
    ct_thresholds = {col: (mean[col] + std[col]) for col in proportion_df.columns}
    
    return ct_thresholds

In [9]:
ct_thresholds = get_ct_proportion_thershold(st_adata.obsm['proportions'])

In [10]:
ct_interacting_cell_results_np_m_cs = adata.obsm['ct_interacting_cell_results_np_m_cs_df']

In [11]:
cell_com_m_df = adata.uns['ct_ccc_results']['cell_com_df_m_sig'].copy()
ct_pairs_m_set = set([tuple(x) for x in cell_com_m_df[['Cell Type 1', 'Cell Type 2']].values])

In [26]:
metabolites = ['L-Lactate', 'Sodium_calcium exchange']
metabolite = metabolites[0]

In [27]:
ct_pairs_m_set = set([tuple(x) for x in cell_com_m_df[cell_com_m_df['metabolite'] == metabolite][['Cell Type 1', 'Cell Type 2']].values])

In [28]:
cell_types = list(set(ct for cell_type_pair in ct_pairs_m_set for ct in cell_type_pair))

### DE analysis

In [29]:
metab_cell_types = {
    'L-Lactate': ['Endothelial', 'TAM', 'T-Helper', 'CD8+ T cell', 'Tumor', 'DC'],
    'Sodium_calcium exchange': ['Endothelial', 'TAM', 'T-Helper', 'CD8+ T cell', 'Tumor', 'DC'],
}

In [30]:
de_results_dict = {}
for ct in metab_cell_types[metabolite]:
    print(ct)
    rows = adata.obs_names[adata.obs['cell_type'] == ct]
    cols = [col for col in ct_interacting_cell_results_np_m_cs.columns if (metabolite in col) and (ct in col)]
    selected_spots = ct_interacting_cell_results_np_m_cs.loc[rows, cols][ct_interacting_cell_results_np_m_cs.loc[rows, cols].sum(axis=1) != 0].index
    rest = ct_interacting_cell_results_np_m_cs.loc[rows, cols][ct_interacting_cell_results_np_m_cs.loc[rows, cols].sum(axis=1) == 0].index
    mask = st_adata.obs_names.isin(selected_spots)
    mask2 = st_adata.obs_names.isin(rest)
    
    if '_' in ct:
        ct = ct.replace('_', '/')
        
    if (mask.sum() == 0) or (mask2.sum() == 0):
        continue

    _ = de_genes(
        st_model, mask=mask, mask2=mask2, ct=ct, key=ct
    )

    res = st_adata.uns[ct]["de_results"]
    res['adj_pval'] = multipletests(res["pval"], method="fdr_bh")[1]

    de_results_dict[ct] = res

Endothelial
TAM
T-Helper
CD8+ T cell
Tumor
DC


In [31]:
writer=pd.ExcelWriter(os.path.join(DATA_PATH, f"{metabolite.replace('-', '_').replace(' ', '_')}_Slide_seq_lung_ct_Harreman_no_deconv_DestVI_DE_results.xlsx"))
for key, df in de_results_dict.items():
    df.to_excel(writer,sheet_name=key.replace('/', '_'))
writer.close()

### Enrichment analysis

In [18]:
def calc_enrichment_score(val, n, M):
    
    x = int(val.split('/')[0])
    N = int(val.split('/')[1])

    e_overlap = n*N/M
    stat = np.log2(x/e_overlap)

    return stat

In [19]:
def Enrichment_analysis_pipeline(gene_set_h, genes, background):
    enr = gseapy.enrichr(gene_list=genes,
                        gene_sets=gene_set_h,
                        organism='human',
                        outdir=None,
                        background=background
                        )

    if enr.results.shape[0] == 0:
        return

    enr_results = enr.results.set_index('Term')
    enr_results['Stats'] = enr_results['Overlap'].apply(calc_enrichment_score, args=(len(genes), len(background)))
    enr_results['ngenes'] = enr_results['Genes'].map(lambda x: len(x.split(';')))

    return enr_results

In [20]:
gene_set = 'BioPlanet_2019'
gene_set_h = gseapy.get_library(gene_set)

In [21]:
background = st_adata.var_names.tolist()

In [32]:
metabolites = ['L-Lactate', 'Sodium_calcium exchange']
metabolite = metabolites[0]

In [33]:
de_results_dict = {}
excel_file=pd.read_excel(os.path.join(DATA_PATH, f"{metabolite.replace('-', '_').replace(' ', '_')}_Slide_seq_lung_ct_Harreman_no_deconv_DestVI_DE_results.xlsx"), sheet_name=None, index_col=0)
for key, df in excel_file.items():
    de_results_dict[key] = df

In [34]:
enr_results_dict = {}
for key, df in de_results_dict.items():
    print(key)
    for dir in ['pos', 'neg']:
        genes = df[(df['adj_pval'] < 0.05) & (df['log2FC'] > 0)].index.tolist() if dir == 'pos' else df[(df['adj_pval'] < 0.05) & (df['log2FC'] < 0)].index.tolist()
        if len(genes) == 0:
            continue
        enr_results = Enrichment_analysis_pipeline(gene_set_h, genes, background)
        enr_results = enr_results[enr_results['Adjusted P-value'] < 0.05].sort_values('Odds Ratio', ascending=False).copy()
        enr_results_dict[f'{key}_{dir}'] = enr_results

Endothelial
TAM
T-Helper
CD8+ T cell
Tumor
DC


In [35]:
writer=pd.ExcelWriter(os.path.join(DATA_PATH, f"{metabolite.replace('-', '_').replace(' ', '_')}_Slide_seq_lung_ct_Harreman_no_deconv_DestVI_DE_enrich_analysis.xlsx"))
for key in enr_results_dict:
    enr_results_dict[key].to_excel(writer,sheet_name=key)
writer.close()